In [1]:
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark = SparkSession(sparkContext=sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/24 23:51:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [3]:
# $example on$
lines = spark.read.text("data/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                         rating=float(p[2]), timestamp=int(p[3])))

    
    

In [4]:
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

In [5]:
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     0|      2|   3.0|1424380312|
|     0|      3|   1.0|1424380312|
|     0|      5|   2.0|1424380312|
|     0|      9|   4.0|1424380312|
|     0|     11|   1.0|1424380312|
+------+-------+------+----------+
only showing top 5 rows



In [6]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",coldStartStrategy="drop")


In [8]:
model = als.fit(training)

In [9]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                    predictionCol="prediction")

In [10]:
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.948879688281409


In [11]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)


In [12]:
# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

In [13]:
# $example off$
userRecs.show()
movieRecs.show()
userSubsetRecs.show()
movieSubSetRecs.show()


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{85, 4.3316855},...|
|    10|[{33, 3.575376}, ...|
|     0|[{35, 5.8521013},...|
|     1|[{90, 3.3965433},...|
|    21|[{53, 4.7845592},...|
|    11|[{92, 5.128945}, ...|
|    12|[{55, 5.627646}, ...|
|    22|[{75, 5.30284}, {...|
|     2|[{83, 5.010873}, ...|
|    13|[{12, 4.064616}, ...|
|     3|[{25, 4.4697685},...|
|    23|[{55, 5.117123}, ...|
|     4|[{29, 4.2114444},...|
|    24|[{29, 5.2828274},...|
|    14|[{29, 5.2489023},...|
|     5|[{12, 5.4350953},...|
|    15|[{46, 4.834661}, ...|
|    25|[{25, 4.897312}, ...|
|    26|[{25, 8.854281}, ...|
|     6|[{7, 5.0961113}, ...|
+------+--------------------+
only showing top 20 rows



+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{17, 5.0345144},...|
|     40|[{2, 3.9393625}, ...|
|     10|[{23, 4.005292}, ...|
|     50|[{23, 4.1255255},...|
|     80|[{22, 5.1170826},...|
|     70|[{8, 2.8972762}, ...|
|     60|[{22, 3.6179614},...|
|     90|[{18, 5.797745}, ...|
|     30|[{11, 5.1025567},...|
|      0|[{10, 2.7572548},...|
|     31|[{17, 5.19856}, {...|
|     81|[{18, 2.9756026},...|
|     91|[{25, 3.017788}, ...|
|      1|[{17, 4.7343554},...|
|     41|[{26, 3.8548021},...|
|     61|[{17, 3.0230694},...|
|     51|[{22, 4.739072}, ...|
|     21|[{16, 2.382642}, ...|
|     11|[{16, 4.443137}, ...|
|     71|[{25, 3.4805229},...|
+-------+--------------------+
only showing top 20 rows

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{25, 8.854281}, ...|
|    19|[{29, 4.961192}, ...|
|    29|[{46, 5.16579}, {...|
+------+--------------------+

+-------+----------

In [14]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x10f3d30d0>>

23/03/25 01:28:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1047298 ms exceeds timeout 120000 ms
23/03/25 01:28:49 WARN SparkContext: Killing executors is not supported by current scheduler.
